In [58]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as random
from scipy.signal import convolve
import scipy
from joblib import Parallel, delayed
import time
from scipy import sparse
from tqdm import tqdm, tqdm_notebook

num_cores = 32

%run initMethods.py
%run makeGif.py
%run methods_parallel.py

In [75]:
params = { #parameters relevant for the equations
    "Nh":          1000000,
    "N0":           100000,
    "R0":              20,
    "M":                1, #Also L, total number of spacers
    "D":                3, #Unused
    "mu":             0.02, #mutation rate
    "v0":              10,
    "gamma_shape":     20, 
    "Np":               0, #Number of Cas Protein
    "dc":               3, #Required number of complexes to activate defence
    "h":                4, #coordination coeff
    "r":              1000, #cross-reactivity kernel
    "rho":            5e-5, #spacer sharing coeff
}
sim_params = { #parameters relevant for the simulation (including Inital Valuess)
    "xdomain":                  200,
    "dx":                         1,
    "t0":                         0, 
    "tf":                       100,
    "dt":                         1,
    "initial_mean_n":           [0,0],
    "initial_var_n":               10,
    "initial_mean_nh":           [0,0],
    "initial_var_nh":               10,
    "n_step_prior":               5,
    "conv_size":               1000,
    "num_threads":               4,
    "tail_axis":            [1, 1],
}

In [76]:
n = scipy.sparse.dok_matrix(
    init_guassian_parallel(params["N0"], sim_params))

In [77]:
nh = scipy.sparse.dok_matrix(
    init_uniform_parallel(params["Nh"], sim_params))

In [78]:
kernel = init_quarter_kernel(params, sim_params)

In [81]:
x_ind_nh, y_ind_nh = nh.nonzero()
x_ind_n, y_ind_n = n.nonzero()
conv_size = sim_params["conv_size"]
xdomain = sim_params["xdomain"]
Nh = params["Nh"]
M = params["M"]
num_threads = sim_params["num_threads"]

ind_nh_subsets = zip(np.array_split(x_ind_nh, num_threads), np.array_split(y_ind_nh, num_threads))
ind_n_subsets = zip(np.array_split(x_ind_n, num_threads), np.array_split(y_ind_n, num_threads))

def convolve_subset(ind_nh, ind_n):
    res = scipy.sparse.dok_matrix(nh.shape, dtype=int)
    for x_nh, y_nh in tqdm(zip(x_ind_nh, y_ind_nh), total=len(x_ind_nh)):
        for x_n, y_n in zip(x_ind_n, y_ind_n):

            x_kernel = np.abs(x_nh-x_n)
            y_kernel = np.abs(y_nh-y_n)

            if np.any((x_kernel >= conv_size, y_kernel>= conv_size)):
                continue
            
            value = nh[x_nh, y_nh]/Nh
            
            try:
                interaction = kernel[x_kernel, y_kernel]
            except(IndexError):
                print("wtf? Convolution out of Bounds??", x_kernel, y_kernel)

                break

            res[x_n, y_n]+= value*interaction
    return res

results = Parallel(n_jobs=num_cores)(delayed(convolve_subset)
    (ind_nh_subset, ind_n_subset) for ind_nh_subset, ind_n_subset 
        in zip(ind_nh_subsets, ind_n_subsets))
